In [1]:
%load_ext autoreload
%autoreload 2

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/Results
#MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
#IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
#%cd /content/Generative_Models/
#!pip -q install -r /content/Generative_Models/requirements.txt
#!pip install colorlog==4.0.2
#%cd /content/

# Connect Drive to save and restore results

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/Results
#MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
#IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well

# Settings

In [4]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [5]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [6]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# only on colab

In [7]:
#!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [8]:
#!tar -xvf data/.pokemon/pokemon_combined.tar --directory data/.pokemon/

# Dataset loading

In [9]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

Using TensorFlow backend.


In [10]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [11]:
from transformation.lmdb_transformer import LmdbTransformer

In [12]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

  DEBUG    | Looking for images in '_training'
  DEBUG    | Looking for images in '_training'


  WARNING  | No files found
  WARNING  | No files found
C:\Users\Khalid\Documents\projects\Generative_Models\transformation\file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


  DEBUG    | Looking for images in '_validation'
  DEBUG    | Looking for images in '_validation'


  WARNING  | No files found
  WARNING  | No files found


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [13]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [14]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [15]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [16]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [17]:
latents_dim

100

In [18]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [19]:
model_name = dataset_name+'AE_Dense_reconst_psnr'
experiments_dir='experiments'+sep_local+model_name

In [20]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [21]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_psnr\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1666)              168266    
_________________________________________________________________
dense_5 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_6 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             50010000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

  WARNING  | None
  WARNING  | None


In [27]:
from evaluation.quantitive_metrics.peak_signal_to_noise_ratio import prepare_psnr
#from statistical.losses_utilities import similarty_to_distance
from statistical.ae_losses import expected_loglikelihood_with_lower_bound as ellwlb

In [28]:
#ae.compile(metrics=None)
#ae.compile()
ae.compile(loss={'x_logits': lambda x_true, x_logits: prepare_psnr([ae.batch_size]+ae.get_inputs_shape())(x_true, x_logits)})

Model: "pokemonAE_Dense_reconst_psnr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 100)               50003250  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       55742822  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 105,746,072
Trainable params: 105,745,866
Non-trainable params: 206
_________________________________________________________________
None


In [29]:
ae.output_shape

(None, 100, 100, 3)

# Callbacks

In [30]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [31]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [32]:
ms = ModelSaver(filepath=_restore)

In [33]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_psnr\\csv_dir\\pokemonAE_Dense_reconst_psnr.csv'

In [34]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [35]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [36]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 3.36437, saving model to experiments\pokemonAE_Dense_reconst_psnr\var_save_dir
5000/5000 - 1458s - loss: 3.3644 - psnr: 3.1149 - total_variation: 97410.6328 - ssim_multiscale: 0.0585 - sharpdiff: 4.6627 - mean_absolute_error: 0.6382 - mean_squared_error: 0.4980 - val_loss: 2.2989 - val_psnr: 1.9932 - val_total_variation: 15743.4863 - val_ssim_multiscale: 0.2181 - val_sharpdiff: 9.1672 - val_mean_absolute_error: 0.7428 - val_mean_squared_error: 0.6374
Epoch 2/1000000


KeyboardInterrupt: 

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=100)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
is_mean.compute()

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)